# PolrModels.jl

PolrModels.jl provides Julia utilities to fit ordered multinomial models.

## Installation

This package requires Julia v0.7 or later. The package has not yet been registered and must be installed using the repository location. Start julia and use the `]` key to switch to the package manager REPL
```julia
(v0.7) pkg> add https://github.com/Hua-Zhou/PolrModels.git#v0.7
```

In [1]:
versioninfo()

Julia Version 0.7.0
Commit a4cb80f3ed (2018-08-08 06:46 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin14.5.0)
  CPU: Intel(R) Core(TM) i7-6920HQ CPU @ 2.90GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-6.0.0 (ORCJIT, skylake)
Environment:
  JULIA_EDITOR = code


In [2]:
using PolrModels, BenchmarkTools

## Example data

`housing` is a data set from R package `MASS`. The outcome of interest is `Sat` (satisfication) that takes values `Low`, `Medium`, or `High`. Predictors include `Infl` (inflation, categorical), `Type` (housing type, categorical), and `Cont` (categorical). `Freq` codes number of observation for each combination of levels.

In [3]:
using RDatasets

housing = dataset("MASS", "housing")

,Sat,Infl,Type,Cont,Freq
,Categorical…,Categorical…,Categorical…,Categorical…,Int32
1,Low,Low,Tower,Low,21
2,Medium,Low,Tower,Low,21
3,High,Low,Tower,Low,28
4,Low,Medium,Tower,Low,34
5,Medium,Medium,Tower,Low,22
6,High,Medium,Tower,Low,36
7,Low,High,Tower,Low,10
8,Medium,High,Tower,Low,11
9,High,High,Tower,Low,36


Total number of observations is

In [4]:
sum(housing[:Freq])

1681

## Fit ordered multinomial models

### Proportional odds model

To fit an ordered multinomial model using default link `link=LogitLink()`, i.e., proportional odds model

In [5]:
house_po = polr(@formula(Sat ~ 0 + Infl + Type + Cont), housing; wts = housing[:Freq])

StatsModels.DataFrameRegressionModel{PolrModel{Int64,Float64,LogitLink},Array{Float64,2}}

Formula: Sat ~ Infl + Type + Cont

Coefficients:
      Estimate Std.Error  t value Pr(>|t|)
θ1   -0.496141  0.124847 -3.97398   0.0002
θ2    0.690706  0.125472  5.50486    <1e-6
β1    0.566392  0.104653   5.4121    <1e-6
β2     1.28881  0.127156  10.1357   <1e-14
β3   -0.572352  0.119238 -4.80008    <1e-5
β4   -0.366182  0.155173 -2.35983   0.0213
β5    -1.09101  0.151486 -7.20206    <1e-9
β6    0.360284 0.0955358   3.7712   0.0004


!!! note

    It is necessary to **exclude intercept** in formula because ordered multinomial model automatically includes intercept for properly modeling.

!!!

Since there are $J=3$ categories in `Sat`, the fitted model has two intercept parameters that satisfy $\theta_1 \le \theta_2$. $\beta_j$ are regression coefficients for predictors. $\beta_1, \beta_2$ for `Infl` (3 levels), $\beta_3, \beta_4, \beta_5$ for `Type` (4 levels), and $\beta_6$ for `Cont` (2 levels). 

Deviance (-2 loglikelihood) of the fitted model is

In [6]:
deviance(house_po)

3479.149299072586

Estimated regression coefficients are

In [7]:
coef(house_po)

8-element Array{Float64,1}:
 -0.49614061341454607
  0.6907056439400201 
  0.5663915864515744 
  1.2888108250034271 
 -0.5723518911944651 
 -0.3661822718560424 
 -1.091011574737562  
  0.3602844417528419 

with standard errors

In [8]:
stderror(house_po)

8-element Array{Float64,1}:
 0.12484725578571429
 0.12547194322484329
 0.10465278622775005
 0.12715610075413825
 0.1192380112188433 
 0.15517334323365534
 0.15148598827529147
 0.09553578572175248

### Ordered probit model

To fit an ordered multinomial model using default link `link=ProbitLink()`, i.e., ordered probit model

In [9]:
house_op = polr(@formula(Sat ~ 0 + Infl + Type + Cont), housing, ProbitLink(); wts = housing[:Freq])

StatsModels.DataFrameRegressionModel{PolrModel{Int64,Float64,ProbitLink},Array{Float64,2}}

Formula: Sat ~ Infl + Type + Cont

Coefficients:
      Estimate Std.Error  t value Pr(>|t|)
θ1   -0.299829 0.0761537 -3.93716   0.0002
θ2     0.42672 0.0764043  5.58502    <1e-6
β1    0.346423 0.0641371  5.40129    <1e-5
β2    0.782914 0.0764262  10.2441   <1e-14
β3   -0.347538 0.0722909 -4.80749    <1e-5
β4   -0.217889 0.0947661 -2.29923   0.0248
β5   -0.664175    0.0918 -7.23502    <1e-9
β6    0.222386 0.0581227  3.82616   0.0003


In [10]:
deviance(house_op)

3479.6888425652414

### Proportional hazards model

To fit an ordered multinomial model using default link `link=CloglogLink()`, i.e., proportional hazards model

In [11]:
house_ph = polr(@formula(Sat ~ 0 + Infl + Type + Cont), housing, CloglogLink(); wts = housing[:Freq])

StatsModels.DataFrameRegressionModel{PolrModel{Int64,Float64,CloglogLink},Array{Float64,2}}

Formula: Sat ~ Infl + Type + Cont

Coefficients:
      Estimate Std.Error  t value Pr(>|t|)
θ1   -0.796225 0.0896499 -8.88149   <1e-12
θ2   0.0553535 0.0855972 0.646674   0.5202
β1    0.382045 0.0702599  5.43759    <1e-6
β2    0.915384 0.0925608  9.88954   <1e-13
β3   -0.407228 0.0860718 -4.73125    <1e-4
β4    -0.28056   0.11115 -2.52416   0.0141
β5   -0.742481  0.101331 -7.32728    <1e-9
β6    0.209235 0.0651057  3.21377   0.0021


In [12]:
deviance(house_ph)

3484.0531705626913

From the deviances, we see the proportional odds model (logit link) has the best fit among all three models.

In [13]:
@show deviance(house_po), deviance(house_op), deviance(house_ph)

(deviance(house_po), deviance(house_op), deviance(house_ph)) = (3479.149299072586, 3479.6888425652414, 3484.0531705626913)


(3479.149299072586, 3479.6888425652414, 3484.0531705626913)

### Alternative syntax without using DataFrame

An alternative syntax is useful when it is inconvenient to use DataFrame
```julia
polr(X, y, link, solver; wts)
```
where `y` is the response vector and `X` is the `n x p` predictor matrix **excluding** intercept.

## Optimization algorithms

PolrModels.jl relies on nonlinear programming (NLP) optimization algorithms to find the maximum likelihood estimate (MLE). User can input any solver supported by the MathProgBase.jl package (see <http://www.juliaopt.org>) as the 4th argument of `polr` function. Common choices are:  
- Ipopt solver: `IpoptSolver(print_level=0)`. See [Ipopt.jl](https://github.com/JuliaOpt/Ipopt.jl) for numerous arugments to `IpoptSolver`. For example, setting `print_level=5` is useful for diagnosis purpose.   
- [NLopt package](https://github.com/JuliaOpt/NLopt.jl): `NLoptSolver(algorithm=:LD_SLSQP)`, NLoptSolver(algorithm=:LD_LBFGS). See [NLopt algorithms](https://nlopt.readthedocs.io/en/latest/NLopt_Algorithms/) for all algorithms in NLopt.jl.

When optimization fails, user can always try another algorithm.

Use Ipopt (interior-point) solver

In [14]:
polr(@formula(Sat ~ 0 + Infl + Type + Cont), housing, LogitLink(), IpoptSolver(print_level=3); wts = housing[:Freq])


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

Total number of variables............................:        8
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0


Number of Iterations....: 38

    

StatsModels.DataFrameRegressionModel{PolrModel{Int64,Float64,LogitLink},Array{Float64,2}}

Formula: Sat ~ Infl + Type + Cont

Coefficients:
      Estimate Std.Error  t value Pr(>|t|)
θ1   -0.496135  0.124847 -3.97394   0.0002
θ2    0.690708  0.125472  5.50488    <1e-6
β1    0.566394  0.104653  5.41212    <1e-6
β2     1.28882  0.127156  10.1357   <1e-14
β3    -0.57235  0.119238 -4.80006    <1e-5
β4   -0.366186  0.155173 -2.35985   0.0213
β5    -1.09101  0.151486 -7.20208    <1e-9
β6    0.360284 0.0955358  3.77119   0.0004


Use SLSQP (sequential quadratic programming) in NLopt.jl package

In [15]:
polr(@formula(Sat ~ 0 + Infl + Type + Cont), housing, LogitLink(), NLoptSolver(algorithm=:LD_SLSQP); wts = housing[:Freq])

StatsModels.DataFrameRegressionModel{PolrModel{Int64,Float64,LogitLink},Array{Float64,2}}

Formula: Sat ~ Infl + Type + Cont

Coefficients:
      Estimate Std.Error  t value Pr(>|t|)
θ1   -0.496141  0.124847 -3.97398   0.0002
θ2    0.690706  0.125472  5.50486    <1e-6
β1    0.566392  0.104653   5.4121    <1e-6
β2     1.28881  0.127156  10.1357   <1e-14
β3   -0.572352  0.119238 -4.80008    <1e-5
β4   -0.366182  0.155173 -2.35983   0.0213
β5    -1.09101  0.151486 -7.20206    <1e-9
β6    0.360284 0.0955358   3.7712   0.0004


Use LBFGS (quasi-Newton algorithm) in NLopt.jl package

In [16]:
polr(@formula(Sat ~ 0 + Infl + Type + Cont), housing, LogitLink(), NLoptSolver(algorithm=:LD_LBFGS); wts = housing[:Freq])

StatsModels.DataFrameRegressionModel{PolrModel{Int64,Float64,LogitLink},Array{Float64,2}}

Formula: Sat ~ Infl + Type + Cont

Coefficients:
      Estimate Std.Error  t value Pr(>|t|)
θ1   -0.496111  0.124847 -3.97375   0.0002
θ2    0.690732  0.125472  5.50507    <1e-6
β1    0.566394  0.104653  5.41212    <1e-6
β2     1.28882  0.127156  10.1357   <1e-14
β3   -0.572352  0.119238 -4.80008    <1e-5
β4    -0.36616  0.155173 -2.35968   0.0214
β5    -1.09102  0.151486 -7.20212    <1e-9
β6    0.360319 0.0955359  3.77156   0.0004


## Likelihood ratio test (LRT)

`polr` function calculates the Wald test (or t-test) p-value for each predictor in the model.

Step 1: Fit the null model with only `Infl` and `Type` factors.

In [17]:
house_null = polr(@formula(Sat ~ 0 + Infl + Type), housing; wts = housing[:Freq])

StatsModels.DataFrameRegressionModel{PolrModel{Int64,Float64,LogitLink},Array{Float64,2}}

Formula: Sat ~ Infl + Type

Coefficients:
      Estimate Std.Error  t value Pr(>|t|)
θ1   -0.672949  0.115689  -5.8169    <1e-6
θ2    0.505629  0.115203  4.38903    <1e-4
β1    0.548392  0.104308  5.25743    <1e-5
β2      1.2373  0.125978  9.82155   <1e-13
β3   -0.521441  0.118215 -4.41094    <1e-4
β4   -0.289347  0.153387 -1.88639   0.0637
β5    -1.01404  0.149562 -6.78008    <1e-8


Step 2: To test significance of the `Cont` variable, we use `polrtest` function. The first argument is the fitted null model, the second argument is the predictor vector to be tested

In [18]:
# last column of model matrix is coding for Cont (2-level factor)
cont = modelmatrix(house_po.model)[:, end]
# calculate p-value
polrtest(house_null, cont; test=:LRT)

0.000155351855453278

## Score test

User can perform **score test** using the `polrtest` function. Score test has the advantage that, when testing a huge number of predictors such as in genomewide association studies (GWAS), one only needs to fit the null model once and then testing each predictor is cheap. Both Wald and likelihood ratio test (LRT) need to fit a separate alternative model for each predictor being tested.

Step 1: Fit the null model with only `Infl` and `Type` factors.

In [19]:
house_null = polr(@formula(Sat ~ 0 + Infl + Type), housing; wts = housing[:Freq])

StatsModels.DataFrameRegressionModel{PolrModel{Int64,Float64,LogitLink},Array{Float64,2}}

Formula: Sat ~ Infl + Type

Coefficients:
      Estimate Std.Error  t value Pr(>|t|)
θ1   -0.672949  0.115689  -5.8169    <1e-6
θ2    0.505629  0.115203  4.38903    <1e-4
β1    0.548392  0.104308  5.25743    <1e-5
β2      1.2373  0.125978  9.82155   <1e-13
β3   -0.521441  0.118215 -4.41094    <1e-4
β4   -0.289347  0.153387 -1.88639   0.0637
β5    -1.01404  0.149562 -6.78008    <1e-8


Step 2: To test significance of the `Cont` variable, we use `polrtest` function. The first argument is the fitted null model, the second argument is the predictor vector to be tested

In [20]:
# last column of model matrix is coding for Cont (2-level factor)
cont = modelmatrix(house_po.model)[:, end]
# calculate p-value
polrtest(house_null, cont; test=:score)

0.00015872510523744766

Step 3: Now suppose we want to test significance of another predictor, `z1`. We just need to call `polrtest` with `z1` and the same fiited null model. No model fitting is needed.

For demonstration purpose, we generate `z1` randomly. The score test p-value of `z1` is, not suprisingly, large.

In [21]:
z1 = randn(nobs(house_null))
polrtest(house_null, z1)

3.223037126997464e-37

Step 4: We can also test a set of precitors or a factor.

In [22]:
z3 = randn(nobs(house_null), 3)
polrtest(house_null, z3)

0.5274404103882856